# Step 1: Import pandas as pd

In [59]:
import pandas as pd
census = pd.read_csv('census_data.csv')
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [60]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [61]:
def label_fix(label):
    if label == ' <=50K':
        return 0
    else:
        return 1

In [62]:
# can be able to do the fucntion above using lambda as well

In [63]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

In [64]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


# Step 2 : Perfrom train/test  Split

In [65]:
X = census.drop('income_bracket',axis=1)
y = census['income_bracket']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

# Step 3: Create Feature Columns

In [66]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [67]:
import tensorflow as tf

In [68]:
# Create Feature Column for Categorical columns

In [69]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

In [70]:
# Create Feature Column for Numeric columns

In [71]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [72]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

# Step 4: Create Input Function

In [73]:
input_func = tf.estimator.inputs.pandas_input_fn(x= X_train,
                                                y=y_train, 
                                                batch_size=100,
                                                num_epochs=None,
                                                shuffle=True)

# Step 5 : Create Model 

In [74]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\DHINAG~1.A\\AppData\\Local\\Temp\\tmpa1or2y5z', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F61A121CF8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Step 6: Train the Model

In [75]:
model.train(input_fn=input_func,steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\DHINAG~1.A\AppData\Local\Temp\tmpa1or2y5z\model.ckpt.
INFO:tensorflow:loss = 69.31474, step = 1
INFO:tensorflow:global_step/sec: 117.005
INFO:tensorflow:loss = 269.32382, step = 101 (0.859 sec)
INFO:tensorflow:global_step/sec: 289.052
INFO:tensorflow:loss = 219.28726, step = 201 (0.342 sec)
INFO:tensorflow:global_step/sec: 282.896
INFO:tensorflow:loss = 770.6892, step = 301 (0.353 sec)
INFO:tensorflow:global_step/sec: 288.524
INFO:tensorflow:loss = 253.47507, step = 401 (0.351 sec)
INFO:tensorflow:global_step/sec: 283.987
INFO:tensorflow:loss = 438.97827, step = 501 (0.352 sec)
INFO:tensorflow:global_step/sec: 282.737
INFO:tensorflow:loss = 105.047966, step = 601 (0.355 sec)
INFO:tensorflow:global_s

# Step 7 : Evaluate and Predictions

In [76]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [77]:
pred_gen = model.predict(input_fn=pred_fn)

In [78]:
predictions = list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\DHINAG~1.A\AppData\Local\Temp\tmpa1or2y5z\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [79]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [80]:
from sklearn.metrics import classification_report

In [81]:
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90      8161
           1       0.69      0.68      0.68      2585

   micro avg       0.85      0.85      0.85     10746
   macro avg       0.79      0.79      0.79     10746
weighted avg       0.85      0.85      0.85     10746

